## Code équipe L4 (groupe 7+11) sur le sujet 2

Requirements

In [ ]:
!pip install -q https://github.com/pyannote/pyannote-audio/archive/develop.zip
!pip install -q speechbrain
!pip install pydub
!pip install tensorflow_io

     / 13.6 MB 350 kB/s
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 77 kB 3.6 MB/s 
     |████████████████████████████████| 74 kB 3.2 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
     |████████████████████████████████| 41 kB 437 kB/s 
     |████████████████████████████████| 49 kB 4.2 MB/s 
     |████████████████████████████████| 582 kB 49.6 MB/s 
     |████████████████████████████████| 109 kB 52.1 MB/s 
     |████████████████████████████████| 408 kB 49.7 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 43.1 MB/s 
     |████████████████████████████████| 112 kB 42.0 MB/s 
     |████████████████████████████████| 130 kB 48.3 MB/s 
     |████████████████████████████████| 308 kB 50.8 MB/s 
     |████████████████████████████████| 210 kB 49.8 MB/s 
     |████████████████████████████████| 81 kB 6.5 

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_hub as hub
from IPython import display
from pydub import AudioSegment
import math
from pyannote.audio import Pipeline
import librosa  # just to demo, not necessary, as you already have the data
import soundfile
from pydub import AudioSegment as am
from pyannote.audio.pipelines import SpeakerDiarization 


## for pre-processing
!apt install ffmpeg
!pip install spleeter
import os 

#Not necessary
from scipy.io import wavfile
import scipy
from pyannote.core import notebook, Timeline


Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
     |████████████████████████████████| 51 kB 3.0 MB/s 
     |████████████████████████████████| 77 kB 3.4 MB/s 
     |████████████████████████████████| 14.8 MB 39.9 MB/s 
     |████████████████████████████████| 183 kB 47.2 MB/s 
     |████████████████████████████    | 397.4 MB 43.8 MB/s eta 0:00:02

Collecting Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r /content/drive/MyDrive/Input/ /content/input/

In [ ]:
!cp -r /content/drive/MyDrive/men_and_women_yamnet/ /content/men_and_women_yamnet/

Defining functions and variables

In [ ]:
class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = os.path.join(folder, filename)
        
        self.audio = AudioSegment.from_wav(self.filepath)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def single_split(self, from_sec, to_sec, split_filename):
        t1 = from_sec * 1000
        t2 = to_sec * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(os.path.join(self.folder, split_filename), format="wav")
        
    def multiple_split(self, sec_per_split=2):
        total_secs = math.ceil(self.get_duration())
        for i in range(0, total_secs, 2):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+sec_per_split, split_fn)
            print(str(i) + ' Done')
            if i == total_secs - sec_per_split:
                print('All splited successfully')

In [ ]:
def convert_to_16k(path):
    '''Convertit le fichier en audio 16000Hz
    et 16-bits'''
    sound = am.from_file(path, format='wav')
    sound = sound.set_frame_rate(16000)
    sound.export('cache_temp/temp_file.wav' , format='wav')
    y, sr = librosa.load(path, sr=16000)
    # write to a new wave file with sample rate sr and format 'signed 16bit'
    soundfile.write(path, y, 16000, subtype='PCM_16')
    os.remove('cache_temp/temp_file.wav')

In [ ]:
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor."""
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [ ]:
saved_model_path = './men_and_women_yamnet'

reloaded_model = tf.saved_model.load(saved_model_path)
classes = ['m', 'f']
cwd = os.path.abspath('')

In [ ]:
def count_speakers(file):
  own_file = file
  OWN_FILE = {'audio': own_file}
  notebook.reset()
  
  pipeline = SpeakerDiarization()
  initial_params = {"onset":0.60, "offset":0.481,"min_duration_on":0.0055,"min_duration_off":0.098, "min_activity":2.078, "stitch_threshold": 0.40, "clustering": {"method": "average", "threshold": 0.1}, }
  pipeline.instantiate(initial_params)
  
  os.system('spleeter separate -o inter/ -f "{instrument}/{filename}.{codec}" '+ file)
  dia = pipeline('/content/inter/'+file[16::])
  
  speakers = dia.labels()
  resu = []
  filemalecount = 0
  filefemalecount = 0
  for i,speaker in enumerate(speakers):
    pre = dia.label_timeline(speaker)
    timel = Timeline()
    folder = os.path.join(os.path.join(cwd, "output"), "speaker_" + str(i))
    is_speaker_male_likelihood = 0
    is_speaker_female_likelihood = 0
    try:
      os.mkdir(folder)
    except:
      pass
    split_wav = SplitWavAudioMubin(folder, file)
    export_filename = os.path.basename(file)
    for segment in pre:
      if segment.duration >= 2.0:
        start = segment.start
        end = segment.end
        is_segment_male_likelihood = 0
        is_segment_female_likelihood = 0
        for i, t in enumerate(np.arange(start, end, 2)):
              split_fn = str(i) + '_' + export_filename
              if i+2 <= end:
                split_wav.single_split(t, t+2, split_fn)
                path_to_analyze = folder + '/' + split_fn
                convert_to_16k(path_to_analyze)
                testing_wav_data = load_wav_16k_mono(path_to_analyze) # Il faut mettre une donnee en 16000Hz
                reloaded_results = reloaded_model(testing_wav_data)
                men_or_women = classes[tf.argmax(reloaded_results)]
                if men_or_women == 'm':
                  is_segment_male_likelihood +=1
                else:
                  is_segment_female_likelihood += 1      
        if is_segment_male_likelihood > is_segment_female_likelihood:
          filemalecount +=1
        else: 
          filefemalecount +=1
  return {'M' : filemalecount, 'F' : filefemalecount}

Code execution

In [ ]:
filelist = [file for file in os.listdir('/content/input/')]
filelistabs = ['/content/input/' + file for file in filelist if file.endswith(".wav")]
df_to_concat = []
for i, file in enumerate(filelistabs):
  dic = count_speakers(file)
  print(file + " :")
  print(dic)
  ##df = df_dic(dic,file)
  ##df_to_concat.append(df)
##df_results = pd.concat(df_to_concat, axis=1)
##df_results.to_csv("results.csv")
